In [1]:
import numpy as np
import pandas as pd
import datetime as dt

%matplotlib inline

pd.options.display.float_format='{:.2f}'.format
pd.options.display.max_rows = 999

df = pd.read_csv('../data/000015', index_col= 'Date', names=['Date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Money', 'PE', 'PB'], parse_dates=True, header=None)
df['Return'] = df['Close'].pct_change() * 100
df['Year'] = [i.year for i in df.index]
df['Month'] = [i.month for i in df.index]
df['Day'] = [i.day for i in df.index]

# Basic indexing

In [ ]:
df_prices = df[['Close', 'High', 'Low']]
   
series_close = df['Close']
close_of_a_day = series_close['2010-01-04']
close_of_a_day

# can't get row of dataframe like:
try:
    df_prices['2010-01-04']
except KeyError:
    pass

## Accessing attributes using dot operator

In [ ]:
df.Close
%matplotlib inline
# select by specifying column indexes
df[[1, 2]]

## Range slicing

The synttax of the slicing operator exactly matches that of NumPy:

```python
ar[startIndex: endIndex: stepValue]
```

where the default values if not specified are as follows:

* 0 for startIndex
* arraysize-1 for endIndex
* 1 for stepValue

## Dropping entries from an axis

In [ ]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj.drop(['d', 'c'])

In [ ]:
df.drop(['High', 'Low'], axis=1)

# Label, integer and mixed indexing

* The `.loc` operator: Allows label-oriented indexing
* The `.iloc` operator: Allows integer-based indexing
* The `.ix` operator: Allows mixed label and integer-based indexing


Gg## Label-oriented indexing

The `.loc` operator supports pure label-based indexing. It accepts the following as valid inputs:

* A single label.
* List or array of labels.
* A slice object with labels.
* A Boolean array.

In [ ]:
df.loc['2010-01-04']

# follows are same
df.loc['2010-01-04', 'Close']
df.loc['2010-01-04']['Close']
df['Close']['2010-01-04']

df.loc[['2010-01-04', '2010-01-05']]
df.loc['2010-01-04': '2010-02-05']

### Selection using a Boolean array

In [ ]:
df.loc[df['Close'] <= df['Close'].min(),:]

## Integer-oriented indexing

The `iloc` operator supports integer-based positional indexing. It accepts the following as inputs:

* A single integer.
* A list or array of integers.
* A slice object with integers.

In [ ]:
df.iloc[0:10,]

## Mixed indexing with the .ix opeator

The `.ix` operator behaves like a mixture of the `.loc` and `.iloc` operators, with the `.loc` behavior taking precedence. It takes the following as possible inputs:

* A single label or integer
* A list of integers or labels
* An integer slice or label slice
* A Boolean array

In [ ]:
df.ix['2010-01-04']
df.ix[['2010-01-04', '2010-01-05']]
df.ix[df.index[-3:]]
df.ix[0]
df.ix[[0, 2]]
df.ix[1: 3]
df.ix[df['Close'] > 4044.6640]

## MultiIndexing

In [ ]:
df1 = df.reset_index()
df1.set_index(['Year', 'Month', 'Day'], inplace=True)

df1.index.get_level_values(0)
df1.index.get_level_values(1)
df1.index.get_level_values(2)

df1.ix[2011, 2]
df1.ix[2011: 2012]
df1.ix[(2011, 1): (2012,2)]

## Swapping and reordering levels

In [ ]:
df_swapped = df1.swaplevel(0, 1, axis=0)
df_swapped.sortlevel(0).ix[(1,2010):(1,2011)]

# recorder_levels function is more general

df_recorded = df1.reorder_levels(['Month', 'Day', 'Year'], axis = 0)

## Cross sections

The `xs` method provides a shortcut means of selecting data based on a particular index level value.

In [ ]:
df1.xs(2, level='Month')

# same as 

df1.swaplevel(0, 1, axis=0).ix[2]

## Reindexing

In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)

obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])

states = ['Texas', 'Utah', 'California']
frame.reindex(index=['a', 'b', 'c', 'd'], method='ffill', columns=states)

# reindex can be done by label-indexing with ix
frame.ix[['a', 'b', 'c', 'd'], states]

# Boolean indexing

## isin and any all methods

In [ ]:
df.loc[df.index.isin(['2010-01-04', '2010-01-05'])]
df.loc[(df.astype('int') == 2656).any(axis = 1)]

## using the where() method

The `where` method is used to ensure that the result of Boolean filtering is the same shape as the original data.

In [ ]:
df[df['Close'] > 2800]
df.where(df > 2800)

## Operations on indexes

In [ ]:
df1 = df.reset_index()
df1.set_index('Date')

# Function application and mapping

In [ ]:
df.apply(lambda x: x.max() - x.min())
df.apply(lambda x: x.max() - x.min(), axis=1)

In [ ]:
df.apply(lambda x: pd.Series([x.min(), x.max()], index=['min', 'max']))

In [ ]:
# Element-wise apply
df.applymap(lambda x: '%.2f' % x)

The reason for the name `applymap` is that Series has a `map` method for applying an element-wise function.

In [ ]:
df['Close'].map(lambda x: '%.2f' % x)

# Panel data

In [ ]:
prices = get_price(indexes, count = 240, end_date = t.today_str, fields=['high', 'close', 'low'])

prices.swapaxes('items', 'minor')

prices.to_frame().unstack()

# Sorting and ranking

In [ ]:
df.sort_index(axis=1, ascending=False)

df['Close'].sort_values()

df.sort_values(by=['Year', 'Month'])

df['Close'].rank(method='first')

# Grouping of data

## GroupBy mechanics

### The groupby operation

The `groupby` operation can be thought of as part of a process that involves the following three steps:

* Splitting the dataset
* Analyzing the data
* Aggregating or combining the data

The result of a `groupby` operation is not a DataFrame but `dict` of DataFrame objects.

In [ ]:
df['Year'] = [i.year for i in df.index]
df['Month'] = [i.month for i in df.index]
df['Day'] = [i.day for i in df.index]

df_group_by_year = df.groupby('Year')
type(df_group_by_year)
len(df_group_by_year)
df_group_by_year.size().sort_values(ascending=False)

df_group_by_year_month = df.groupby(['Year', 'Month'])
df_group_by_year_month.size().sort_values(ascending=False)

df_group_by_year = df.groupby(lambda x: x.year)
#for name, group in df_group_by_year:
#    print(name)
#    print(group)

df_index_year_month = df.reset_index()
df_index_year_month = df_index_year_month.set_index(['Year', 'Month'])
df_group_by_year = df_index_year_month.groupby(level=['Year', 'Month'])
df_group_by_year.size()

### Iterating over groups

In [ ]:
for (year, month), group in df.groupby(['Year', 'Month']):
    print(year, month)
    print(group)

In [ ]:
pieces = dict(list(df.groupby('Year')))
pieces[2010]

### Selecting a column or subset of columns

In [ ]:
df.groupby(['Year', 'Month'])[['Close']].mean()

### Grouping with dicts and series

In [ ]:
people = pd.DataFrame(np.random.randn(5, 5),
    columns=['a', 'b', 'c', 'd', 'e'],
    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan # Add a few NA values

mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

by_column = people.groupby(mapping, axis=1)
by_column.sum()

map_series = pd.Series(mapping)
people.groupby(map_series, axis=1).count()

### Grouping with functions

In [ ]:
people.groupby(len).sum()

key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

### Grouping by index levels

In [ ]:
df_index_year_month = df.reset_index()
df_index_year_month = df_index_year_month.set_index(['Year', 'Month'])

grouped = df_index_year_month.groupby(level='Month')
grouped.mean()
# same as
df_index_year_month.mean(level='Month')

grouped.sum()

## Data Aggregation

In [ ]:
df.groupby('Year')['Close'].quantile(np.arange(0,1.1,0.1)).unstack()
df.groupby('Year')['Close'].describe().unstack()

### Using the aggregate method

Another way to generate summary statistics by using the aggregate method explicitly:

In [ ]:
grouped.aggregate(np.sum)

### Applying multiple functions

In [ ]:
grouped.agg([np.sum, np.mean, np.size])

grouped['Return'].agg({'Size': np.size, 'Total': np.sum, 'Average': np.mean, 'Deviation': np.std, 'Max': np.max, 'Min': np.min})

### Returning aggregated data in "unindexed" form

In [ ]:
df.groupby(['Year'], as_index=False).mean()
# the reulst is possible to obtain by calling reset_index on result

## Group-wise operations and transformations

### The transform() method

The `groupby-transform` function is used to perform transormation operation on a `group` object. For example, we could replace NaN values in the `groupby` object using the `fillna` method. The resulting object after using the transform has the same size as the original `groupby` object.

In [ ]:
grouped.transform(lambda x: x.fillna(x.mean()))

In [ ]:
df.groupby(['Year', 'Month'])['Close'].transform(lambda x: (x - x.mean()) /x)

### Apply: general split-apply-combine

In [ ]:
def top(df, n=5, column='Close'):
    return df.sort_values(by=column)[-n:]

df.groupby('Year').apply(top)

#### Suppressing the group keys

In [ ]:
df.groupby('Year', group_keys=False).apply(top)

### Examples

#### Filling missing values with group-specific values

In [ ]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan

s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
    'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

data.groupby(group_key).mean()

fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

#### Random sampling and permutation

In [ ]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

draw(deck)

get_suit = lambda card: card[-1]

deck.groupby(get_suit, group_keys=False).apply(draw, n=5)

#### Group weighted average and correlation

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
    'data': np.random.randn(8), 'weights': np.random.rand(8)})

grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

```python
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)

by_year.apply(spx_corr)
```

#### Group-wise linear regression

In [ ]:
```python
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])
```

## Filtering

The `filter` method enables to apply filtering on a `groupby` object that results in a subset of the initial object.

In [ ]:
grouped.filter(lambda x: np.all([x[col] > 1640 for col in ['High', 'Close', 'Low']]))

## Quantile and bucket analysis

In [ ]:
closes = df['Close'][-240:]
factor = pd.cut(closes, 10)

def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 
            'count': group.count(), 'mean': group.mean()}

grouped = closes.groupby(factor)
grouped.apply(get_stats).unstack()

# Merging and joining

## Using concat

The `concat` function is used to join multiple pandas data structures along a specified axis and possibly perform union or intersection operations along other axes.

In [ ]:
df1 = df.ix['2010-01-04': '2010-01-10', ['Open']]
df2 = df.ix['2010-01-05': '2010-01-12', ['Close']]
df3 = df.ix['2010-01-06': '2010-01-9', ['High']]

pd.concat([df1, df2, df3], axis=1) # outer join
pd.concat([df1, df2, df3], axis=1, join='inner') # inner join
pd.concat([df1, df2, df3], axis=1, join_axes=[df2.index]) # inner join

## Using append

The `append` function is a simpler version of `concat` that concatenates along `axis=0`.

## Appending a single row to a DataFrame

A DataFrame can be appended a single row by passing a series or dictionary to the `append` method.

In [ ]:
df1 = df.ix['2010-01-04': '2010-01-10', ['Open', 'Close']]

df1.append({'Open': 100, 'Close': 100}, ignore_index=True)

## Combining data with overlap

With DataFrames, `combine_first` can thought of as 'patching' missing dta the the calling object with data from the object passed. 

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan], 'b': [np.nan, 2., np.nan, 6.], 'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})

df1.combine_first(df2)

## SQL-like merging/joining of DataFrame objects

The `merge` function is used to obtain joins of two DataFrame objects similar to those used in SQL database queries. The DataFrame objects are analogous SQL tables.

In [ ]:
df1 = df.ix['2010-01-04': '2010-01-10', ['Open', 'High']]
df2 = df.ix['2010-01-05': '2010-01-12', ['Close', 'High']]

pd.merge(df1, df2, how='inner')
pd.merge(df1, df2, how='outer')
pd.merge(df1, df2, how='left')
pd.merge(df1, df2, how='right', left_index=True, right_index=True)

### The join function

The `DataFrame.join` function is used to combine two DataFrames that have different columns with nothing in common. Essentially, this does a longitudinal join of two DataFrames.

In [ ]:
df1 = df.ix['2010-01-04': '2010-01-10', ['Open']]
df2 = df.ix['2010-01-05': '2010-01-12', ['Close']]

df1.join(df2)

# Pivots and reshaping data

## Stacking and unstacking

### The stack() function

In [ ]:
result = df.stack(dropna=False)
df.unstack('Date') # or df.unstack(0)

## Pivoting "long" to "wide" format

A common way to store multiple time series in databases and CSV is in so-called *long* or *stacked* format.

|        | date                | item    | value       |
| ------ | ------------------- | ------- | ----------- |
| 0      | 1959-03-31 00:00:00 | realgdp | 2710.349    |
| 1      | 1959-03-31 00:00:00 | infl    | 0.000       |
| 2      | 1959-03-31 00:00:00 | unemp   | 5.800       |
| 3      | 1959-06-30 00:00:00 | realgdp | 2778.801    |
| 4      | 1959-06-30 00:00:00 | infl    | 2.340       |
| 5      | 1959-06-30 00:00:00 | unemp   | 5.100       |
| 6      | 1959-09-30 00:00:00 | realgdp | 2775.488    |
| 7      | 1959-09-30 00:00:00 | infl    | 2.740       |
| 8      | 1959-09-30 00:00:00 | unemp   | 5.300       |
| 9      | 1959-12-31 00:00:00 | realgdp | 2785.204    |

DateFrame's `pivot` method performs exactly trasformation:

```python
pivoted = ldata.pivot('date', 'item', 'value')
```

| item       | infl | realgdp  | unemp |
| ---------- | ---- | -------- | ----- |
| date       |      |          |       |
| 1959-03-31 | 0.00 | 2710.349 | 5.8   |
| 1959-06-30 | 2.34 | 2778.801 | 5.1   |
| 1959-09-30 | 2.74 | 2775.488 | 5.3   |
| 1959-12-31 | 0.27 | 2785.204 | 5.6   |
| 1960-03-31 | 2.31 | 2847.699 | 5.2   |

The first two values passed are columns to be used as the row and column index, and finally an optional value column to fill the DataFrame. Suppose there are two value columns that are to be reshape simulataneously:

```python
ldata['value'] = np.random.randn(len(ldata))
```

By ommitting the last argument, a DataFrame with hierarchical columns is got:

```python
pivoted = ldata.pivot('date', 'item')
pivoted['value'][:5]
pivoted['value2'][:5]
```

Note that `pivot` is just shortcut for creating a hierarchical index using `set_index` and reshaping with `unstack`:

```python
unstacked = ldata.set_index(['data', 'item']).unstack('item')
```


In [ ]:
df.pivot_table(['Return'], index=['Year'], columns=['Month'], aggfunc=np.mean, margins=True, fill_value=0)

## Pivot tables

In [ ]:
df.pivot_table('Return', index=['Year'], columns=['Month'], aggfunc=sum, margins=True, fill_value=0)

## Cross-tabulations: crosstab

A cross-tabulation is a special case of a pivot table that computes group frequencies.

In [ ]:
pd.crosstab([df['Year'], df['Month']], df['Day'], margins=True)

## Other methods to reshape DataFrames

### Using the melt function

# Handling Missing data

| Argument            | Description                                                                                                                                                                                                 |
| ------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `dropna`            | Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.                                                                 |
| `fillna`            | Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.                                                                                                           |
| `isnull`            | Return like-type object containing boolean values indicating which values are missing / NA.                                                                                                                 |
| `notnull`           | Negation of `isnull`.

## Filtering out missing data

In [ ]:
df1 = df.asfreq('D')

df1.dropna()
df1[df.notnull()]

## Filling in missing data

In [ ]:
df.fillna(0, inplace=True)
df.fillna({'PE': -100, 'PB': 1000})

df1.fillna(method='ffill', limit=2)
df1.fillna(method='bfill')

# Data transformation

## Removing duplicates

In [ ]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 'k2': [1, 1, 2, 3, 3, 4, 4]})

data.duplicated()
data.drop_duplicates()
data.drop_duplicates(['k1'])
data.drop_duplicates(['k1', 'k2'], keep='last')

## Transforming data using a function or mapping

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                            'corned beef', 'Bacon', 'pastrami', 'honey ham',
                            'nova lox'],
                   'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

data['animal'] = data['food'].map(str.lower).map(meat_to_animal)

#or
data['food'].map(lambda x: meat_to_animal[x.lower()])

## Replacing values

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

data.replace(-999, np.nan)
data.replace([-999, -1000], np.nan)
data.replace([-999, -1000], [np.nan, 0])
data.replace({-999: np.nan, -1000: 0})

## Renaming axis indexes

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

data.index.map(str.upper)
data.rename(index=str.title, columns=str.upper)
data.rename(index={'OHIO': 'INDIANA'},columns={'three': 'peekaboo'})
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

## Discretization and binning

In [ ]:
cats = pd.cut(df['Close'], 10, precision=2)
pd.value_counts(cats)

In [ ]:
cats = pd.qcut(df['Close'], 10, precision=2)
pd.value_counts(cats)

## Detecting and filtering outliers

In [ ]:
np.random.seed(12345)

data = pd.DataFrame(np.random.randn(1000, 4))
col = data[3]

col[np.abs(col) > 3]
data[(np.abs(data) > 3).any(1)]
data[np.abs(data) > 3] = np.sign(data) * 3

## Permutation and random sampling

In [ ]:
data = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
sampler = np.random.permutation(5)
data.take(sampler)
data.take(np.random.permutation(len(data))[:3])

bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)
draws = bag.take(sampler)

## Computing indicator/dummy variables

# Handling time series

## Reading in time series data

### DataOffset and TimeDelta objects

A `DateOffset` object represents a change or offset in time. The key features of a `DataOffset` object are as follows:

* This can be added/subtracted to/from a `datetime` object to obtain a shifted date
* This can be multiplied by an integer (positive or negative) so that the increment can be applied multiple times
* This has the rollforward and rollback methods to move a date forward to the next offset date or backward to the previous offset date

In [ ]:
xmasDay = pd.datetime(2014, 12, 25)
boxingDay = xmasDay + pd.DateOffset(days=1)
today = pd.datetime.now()
today + pd.DateOffset(weeks=1)
today + 2 * pd.DateOffset(years=2, months=6)

lastDay = pd.datetime(2013, 12, 31)
from pandas.tseries.offsets import QuarterBegin
dtoffset = QuarterBegin()
lastDay + dtoffset
dtoffset.rollforward(lastDay)

weekDelta = dt.timedelta(weeks=1)
today + weekDelta

#### Converting between string and datetime

In [ ]:
stamp = dt.datetime(2011, 1, 3)
str(stamp)
value = stamp.strftime('%Y-%m-%d')

dt.datetime.strptime(value, '%Y-%m-%d')

from dateutil.parser import parse
parse('2011-01-03')
parse('Jan 31, 1997 10:45 PM')
parse('6/12/2011', dayfirst=True)

datestrs = ['7/6/2011', '8/6/2011']
pd.to_datetime(datestrs)

## Time series-related instance methods

### Generating data ranges

In [ ]:
pd.date_range('1/1/2000', '12/1/2000', freq='BM')
pd.date_range('5/2/2012 12:56:31', periods=5, normalize=True)

### Shifting(leading and lagging) data

In [ ]:
df['Close'].shift(3)
df['Close'].shift(3, freq='B')

#### Shifting dates with offsets

In [ ]:
from pandas.tseries.offsets import Day, MonthEnd

now = dt.datetime(2017, 1, 1)
now + 3 * Day()
now + MonthEnd(2)

offset = MonthEnd()
offset.rollforward(now)
offset.rollback(now)

df['Close'].groupby(offset.rollforward).mean()
df['Close'].resample('M').mean()

### Frequency conversion

### Resampling of data

In [ ]:
df.resample('W').bfill()

#### Open-High-Low-Close(OHLC) resampling

In [ ]:
df['Close'].resample('M').ohlc()

#### Resampling with groupby

In [ ]:
df['Close'].groupby(lambda x: (x.year, x.month)).mean()

### Aliases for Time Series frequencies

To specify offsets, a number of aliases are available; some of the most commonly used ones are as follows:

* B, BM: This stands for business day, business month. These are the working days of the month, that is, any day that is not a holiday or a weekend.
* D, W, M, Q, A: It stands for calendar day, week, month, quarter, year-end.
* H, T, S, L, U: It stands for hour, minute, second, millisecond, and microsecond.

Suffixes can be applied to the frequency aliases to specify when in a frequency period to start. These are known as anchoring offsets:

* W-Sun, MON,...
* Q-JAN, FEB, ... DEC
* A-JAN, FEB, ... DEC

These offsets can be used as arguments to the `date_range` and `bdate_range` functions as well as constructors for index types such as `PeriodIndex` and `DatetimeIndex`

## Time series concepts and datatypes

When dealing with time series, there are two main concepts: points in time and range, or time spans. In pandas, the former is represented by the Timestamp datatype, which is equivalent to Python's `datetime`. `datetime` datatype is interchangeable with it. The latter (time span) is represented by the Period datatype, which is specific to pandas.

Each of these datatypes has index datatypes associated with them: `DatetimeIndex` for `Timestamp/Datetime` and `PeriodIndex` for `Period`. These index datatypes are basically subtypes of `numpy.ndarray` that contain the corresponding Timestamp and Period datatypes and can be used as indexes for Series and DataFrame objects.

### Period and period arithmetic

In [ ]:
pd.Period('2014', freq='A-May')
pd.Period('2014-6-11')
pd.Period('2014-6-11 11:00', freq='H')

pd.Period('2014-6-11') + 4
pd.Period('2014-6-11 11:00', freq='H') - 48

pd.Period('2014-04', freq='M')-pd.Period('2013-02', freq='M')

#### PeriodIndex

A `PeriodIndex` object, which is an index type for a `Period` object, can be created in two ways:

* From series of `Period` objects using the `period_range` function, an analogue of `date_range`:

In [ ]:
perRng = pd.period_range('02/01/2014','02/06/2014',freq='D')
perRng[:2]

* It can also be done via direct call to the `Period` consturctor:

In [ ]:
JulyPeriod = pd.PeriodIndex(['07/01/2014','07/31/2014'], freq='D')

#### Period frequency conversion

Periods and `PeriodIndex` objects can be converted to another frequency using their `asfreq`method.

In [ ]:
p = pd.Period('2007', freq='A-JUN')

p.asfreq('M', how='start')
p.asfreq('M', how='end')

In [ ]:
rng = pd.period_range('2006', '2009', freq='A-DEC')
ts = pd.Series(np.random.randn(len(rng)), index=rng)

ts.asfreq('M', how='start')
ts.asfreq('B', how='end')

#### Quarterly period frequencies

In [ ]:
p = pd.Period('2012Q4', freq='Q-JAN')
p.asfreq('D', 'start')
p.asfreq('D', 'end')

p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
p4pm.to_timestamp()

In [ ]:
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')
ts = pd.Series(np.arange(len(rng)), index=rng)
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
ts.index = new_rng.to_timestamp()

ts

### Conversion between Time Series datatypes

In [ ]:
worldCupFinal = pd.to_datetime('07/13/2014', errors='raise')
worldCupFinal.to_period('D')

worldCupKickoff = pd.Period('06/12/2014','D')
worldCupDays = pd.date_range('06/12/2014',periods=32, freq='D')
worldCupDays.to_period()

## Moving window functions

In [ ]:
df['Close'].rolling(window=39).mean()

### Exponentially-weighted functions

In [ ]:
df['Close'].ewm(span=39).mean()

### Binary moving window function

In [ ]:
df1 = pd.read_csv('../data/000300', index_col= 'Date', names=['Date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Money', 'PE', 'PB'], parse_dates=True, header=None)
df2 = pd.read_csv('../data/000015', index_col= 'Date', names=['Date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Money', 'PE', 'PB'], parse_dates=True, header=None)

df1['Return'] = df1['Close'].pct_change()
df2['Return'] = df2['Close'].pct_change()

corr = df1['Return'].rolling(window=125, min_periods=100).corr(df2['Return'])
corr.plot()

### Use-defined moving window functions

In [ ]:
from scipy.stats import percentileofscore

score_at_2percent = lambda x: percentileofscore(x, 0.1)
df['Return'].rolling(window=39).apply(score_at_2percent).plot()

## a-summary-of-time-series-related-objects

| Object              | Summary                                                                                                                                                                                                     |
| ------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `datetime.datetime` | Standard Python `datetime` class                                                                                                                                                                            |
| `Timestamp`         | A pandas class derived from `datetime.datetime`                                                                                                                                                             |
| `DatetimeIndex`     | A pandas class and implemented as an immutable `numpy.ndarray` of the `Timestamp/datetime` objects                                                                                                          |
| `Period`            | A pandas class representing a time period                                                                                                                                                                   |
| `timedelta`         | A pandas class expressing the difference between two `datetime.datetime` instances. It is implemented as `datetime.timedelta`                                                                               |
| `relativedelta`     | Implemented as `dateutil.relativedelta`. dateutil is an extension to the standard Python datetime module. It provides extra functionality such as timedeltas that are expressed in units larger than 1 day. |
| `DateOffset`        | A pandas class representing a regular frequency increment. It has similar functionality to `dateutil.relativedelta`.                                                                                        |
